In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

# Higher Order Methods for Solving Differential Equations
## Lecture 14

### Example

In a constant magnetic field $\vec{B}$, particle of electrical charge $q$ moving with velocity $\vec{v}$ experiences a magnetic force $\vec{F_M}$ given by

\begin{align}
\vec{F_M} &= q \vec{v} \times \vec{B}
\end{align}

Predict the motion of a proton as shown in the figure:

![](http://s3.amazonaws.com/answer-board-image/b18ba716-e5f2-4809-8c28-b559175fd1be.bmp)

Expanding the cross-product and applying Newton's Second Law leads to the following equations of motion:

\begin{align}
m \frac{d^2x}{dt^2} &= - q v_y B \\
m \frac{d^2y}{dt^2} &=   q v_x B \\
\;
\end{align}


We can write this set of two, second-order differential equations into four, first-order differential equations:
\begin{align}
\frac{dv_x}{dt} &= - q v_y B / m\\
\frac{dv_y}{dt} &=   q v_x B / m  \\
\frac{dx}{dt} &= v_x \\
\frac{dy}{dt} &= v_y \\
\end{align}

Let's use the following physical parameters:

In [ ]:
v0 = 4.7e6    # initial horizontal velocity, m/s
q = 1.602e-19 # charge, C
m = 1.67e-27  # mass of particle, kg
B = 0.35      # magnetic field, T 

A numerical solution using Euler's method is shown below:

In [ ]:
def solve_1(tmax = 1e-6, dt = 1e-11):  
    
    # Allocate memory for arrays and set to zero
    N = round(tmax/dt)
    x = np.zeros(N)
    y = np.zeros(N)
    vx = np.zeros(N)
    vy = np.zeros(N)
    t = np.zeros(N)

    # Initial values
    x[0] = 0
    y[0] = 0
    vx[0] = v0
    vy[0] = 0
    t[0] = 0
    
    # Calculate the solution
    for i in range(N-1):
        vx[i+1] = vx[i] - q*vy[i]*B/m*dt
        vy[i+1] = vy[i] + q*vx[i]*B/m*dt
        
        x[i+1]  = x[i] + vx[i]*dt
        y[i+1]  = y[i] + vy[i]*dt
        
        t[i+1]  = t[i] + dt

    return x, y, t

Here is the solution over 1 $\mu$s of time:

In [ ]:
fig, axes = plt.subplots(figsize=(12,6))

x, y, t = solve_1(dt=1e-11, tmax=1e-6)

# plot the x and y component vs time
plt.plot(t*1e6, x*100)
plt.plot(t*1e6, y*100)   
plt.title('Trajectory of a proton in a magnetic field')
plt.ylabel('x or y (cm)')
plt.xlabel('t ($\mu$s)')

# plot the trajectory in an inset axis
fig.add_axes([0.68, 0.2, 0.2, 0.4])
plt.plot(x,y)
plt.xlim(-0.18, 0.18)
plt.ylim(-0.04, 0.32)
plt.xlabel('x')
plt.ylabel('y')
plt.xticks([-0.1, 0.0, 0.1])
plt.yticks([ 0.0, 0.1, 0.2])
plt.show()

### Exercise
> In the above cell, change the timestep to `dt = 1e-10` and re-execute the cell.

> Change it again to `dt=1e-9` and re-execute.

> Change the time step back to `dt=1e-11` and re-execute.


So, to simulate only 1 $\mu$s of model time with Euler's method required 100,000 time steps. 

In [ ]:
tmax = 1e-6
dt = 1e-11
N = round(tmax/dt)
print(N)

 For comparison, here is the analytical solution resolved with a decreasing number of points:

In [ ]:
fig, axes = plt.subplots(4,1, figsize=(12,8))

for i, N in enumerate([500, 250, 100, 50]):
    plt.sca(axes[i])
    ω = q*B/m
    t = np.linspace(0, tmax, N)
    x = v0/ω*np.sin(ω*t)
    y = -v0/ω*(np.cos(ω*t)-1)

    plt.plot(t*1e6, x*100, '.-')
    plt.plot(t*1e6, y*100, '.-')
    plt.ylabel('N = {}'.format(N))
    
fig.suptitle('Trajectory of a proton in a magnetic field\nfor different resolutions')
plt.xlabel('t ($\mu$s)')

So even though the solution can be well resolve with only 500, 250, 100, or even 50 points, the numerical solutions required 100,000 points to converge.

We can do better (achieve the same accuracy with less computational effort) by improving our numerical method.

## Runge-Kutta Methods

The goal is to solve

\begin{align}
\frac{dy}{dt} = f(y,t), \quad y(0) = y_0
\end{align}

### Review Euler's method

We can approximate derivative with a **forward difference**

\begin{align}
 \frac{dy}{dt} &= \frac{y(t+\Delta t) - y(t)}{\Delta t} + O(\Delta t) \\
 y(t+\Delta t) &= y(t) + \frac{dy}{dt}\Delta t + O(\Delta t^2) 
\end{align}

since $\frac{dy}{dt} = f(y,t)$ we can write

$$  y(t+\Delta t) = y(t) + f(y,t) \Delta t + O(\Delta t^2) $$

We say that Euler's method is accurage to *first-order* in $\Delta t$.

### Midpoint scheme or Second-Order Runge-Kutta Method

Start with a **central difference** representation for a derivative

\begin{align}
 \frac{dy}{dt} &= \frac{y(t+\Delta t) - y(t-\Delta t)}{2 \Delta t} + O(\Delta t^2) \\
 y(t+\Delta t) &= y(t-\Delta t) + 2 \frac{dy}{dt} \Delta t + O(\Delta t^3) \\
  &= y(t - \Delta t) + 2 f(y(t), t)\Delta t +O(\Delta t^3)
\end{align}

$f(y(t), t)$ is evaluated at the midpoint in time between $t - \Delta t$ and $t+\Delta t$. 

We can rewrite this as

$$ y(t+\Delta t) = y(t) + 2 f\left(y(t + \Delta t /2), t + \Delta t/2\right)\Delta t +O(\Delta t^3)$$

i.e. $\Delta t \rightarrow \Delta t / 2$, then we can shift $t \rightarrow t + \Delta t / 2$

So, if we can estimate 

$$ f\left(y(t + \Delta t /2), t + \Delta t/2\right) $$

we can gain an order in our truncation error. To do this we need to estimate $y(t+\Delta/2)$.

Trick: use an Euler half-step

$$
y ( t + \Delta t / 2) = y(t) + f(y(t), t)  (\Delta t / 2) + O(\Delta t^2)
$$


We can show (through Taylor expansion) that this leads to a final estimate of $y(t + \Delta t)$ that is accurate to *second-order* in $\Delta t$.

### Midpoint Algorithm

1\. Estimate slope at $t$

$$ s_1 = f(y(t), t) $$

2\. Use $s_1$ to estimate the midpoint between $t$ and $t + \Delta t$

\begin{align}
y^* &= y ( t + \Delta t) \\
       &= y(t) + \frac{\Delta t}{2} s_1
\end{align}
 	

3\. Use $y^*$ to get the the slope at the midpoint

$$ s_2 = f( y^*, t +  \frac{\Delta t}{2} ) $$

4\. Use $s_2$ to estimate $y(t + \Delta t)$

$$ y(t + \Delta t) = y(t) + \Delta t s_2 $$

### Midpoint Algorithm

\begin{align}
s_1 &= f(y_i, t_i) \\
y^* &= y_i + \Delta t / 2 s_1 \\
s_2 &= f(y^*, t_i + \Delta t /2) \\
y_{i+1} &= y_i + \Delta t s_2
\end{align}

Although the algorithm takes two steps, there is an order gain in accuracy. The error of the scheme is of order  $\Delta t^2$. This means that decreasing the timestep  $\Delta t$ by a factor of  2 decreases the error by a factor of 4. 

It also means that compared to the Euler scheme and for the same time step used the accuracy of the scheme is better.

### Fourth-Order Runge-Kutta Method

A very common scheme is the classical fourth-order Runge-Kutta formula.

	
\begin{align}
    s_1 &= f(y_i, t_i,) \\
    y_{1/2} &= y_i + s_1 \Delta t \\ 
    s_2 &= f(y_{1/2}, t + \Delta t / 2) \\
    y_{1/2}^* &= y_i + s_2 \Delta t / 2  \\
    s_3 &= f(y_{1/2}^*, t + \Delta t / 2) \\
    y^* &= y_i + s_3 \Delta t  \\
    s_4 &= f(y^*, t + \Delta t) \\
    y_{i+1} &= y_i  +  1/6 (s_1+2 s_2+2s_3+s_4)\Delta t	\\
\end{align}


The error of the scheme is of order  $\Delta t^4$. This means that decreasing the timestep  $\Delta t$ by a factor of 2 decreases the error by a factor of 16. It also means that compared to the Euler or midpoint schemes the accuracy is typically much better.

## Examples

### Example 1

Solve numerically the following ODE

$$ \frac{dy}{dt} = - y $$

with initial condition

$$ y(0)=1$$

using the Euler and midpoint and RK4 methods and compare graphically the numerical solutions to the exact solution. Take 
$\Delta t$=0.5 and carry out the simulation up to $t=4$.

In [ ]:
tmax = 4
dt = 0.5

N = round(tmax/dt)
t = np.arange(0, tmax, dt)

# Exact solution
y_exact = 1 * np.exp(-t)

# Euler's method
y = np.zeros(N)
y[0] = 1
for i in range(N-1):
    s1 = -y[i]
    y[i+1]  = y[i] + s1*dt
y_euler = y

# Midpoint method
y = np.zeros(N)
y[0] = 1
for i in range(N-1):
    s1 = -y[i]
    ytmp = y[i] + dt/2 * s1
    s2 = -ytmp
    y[i+1]  = y[i] + s2*dt
y_midpoint = y

# RK4 method
y = np.zeros(N)
y[0] = 1
for i in range(N-1):
    s1 = -y[i]
    y1 = y[i] + s1 * dt/2
    s2 = -y1
    y2 = y[i] + s2 * dt/2
    s3 = -y2
    y3 = y[i] + s3 * dt
    s4 = -y3
    y[i+1] = y[i] + (s1 + 2*s2 + 2*s3 + s4)/6*dt
y_RK4 = y

### Compare the solutions

In [ ]:
fig, axes = plt.subplots(figsize=(7,7))
plt.plot(t, y_exact, 'k-',  label='Exact')
plt.plot(t, y_euler, 'o:',  label='Euler')
plt.plot(t, y_midpoint, 'o:',  label='Midpoint')
plt.plot(t, y_RK4, 'o:',  label='RK4')
plt.legend()
plt.xlabel('t')
plt.ylabel('y')
plt.show()

### Example 2

Solve numerically the following ODE

$$ \frac{dy}{dt} = -2 y +3 e^{-4t} $$

 with initial condition
 
$$ y(0)=1$$ 

 using the Euler and midpoint methods, and compare graphically the numerical solutions to the exact solution. Take 
$\Delta t =0.2$ and carry out the simulation up to  $t=3$.

In [ ]:
tmax = 3
dt = 0.2

N = round(tmax/dt)
t = np.arange(0, tmax, dt)

# Exact solution
y_exact = 1/2* np.exp(-4*t)*(5*np.exp(2*t)-3)

# define the right hand side
def f(y, t):
    return -2*y+3*np.exp(-4*t)

# Euler's method
y = np.zeros(N)
y[0] = 1
for i in range(N-1):
    s1 = f(y[i], t[i])
    y[i+1]  = y[i] + s1*dt
y_euler = y

# Midpoint scheme
y = np.zeros(N)
y[0] = 1
for i in range(N-1):
    s1 = f(y[i], t[i])
    ytmp = y[i] + dt/2 * s1
    s2 = f(ytmp, t[i])
    y[i+1]  = y[i] + s2*dt
y_midpoint = y

# RK4 method
y = np.zeros(N)
y[0] = 1
for i in range(N-1):
    s1 = f(y[i], t[i])
    y1 = y[i] + s1 * dt/2
    s2 = f(y1, t[i] + dt/2)
    y2 = y[i] + s2 * dt/2
    s3 = f(y2, t[i] + dt/2)
    y3 = y[i] + s3 * dt
    s4 = f(y3, t[i] + dt)
    y[i+1] = y[i] + (s1 + 2*s2 + 2*s3 + s4)/6*dt
y_RK4 = y

In [ ]:
fig, axes = plt.subplots(figsize=(7,7))
plt.plot(t, y_exact, 'k-',  label='Exact')
plt.plot(t, y_euler, 'o:',  label='Euler')
plt.plot(t, y_midpoint, 'o:',  label='Midpoint')
plt.plot(t, y_RK4, 'o:',  label='RK4')
plt.legend()
plt.xlabel('t')
plt.ylabel('y')
plt.show()

It is clear that for the same time step, the fourth-order Runge-Kutta method is much more accurate.

### Example 3
Let's return the example from Test 1 on a the trajectory of proton in a magnetic field.

\begin{align}
\frac{dv_x}{dt} &= - q v_y B / m\\
\frac{dv_y}{dt} &=   q v_x B / m  \\
\frac{dx}{dt} &= v_x \\
\frac{dy}{dt} &= v_y \\
\end{align}

In [ ]:
def solve_2(tmax = 1e-6, dt = 1e-11):  
    
    # Allocate memory for arrays and set to zero
    N = round(tmax/dt)
    x = np.zeros(N)
    y = np.zeros(N)
    vx = np.zeros(N)
    vy = np.zeros(N)
    t = np.zeros(N)

    # Initial values
    x[0] = 0
    y[0] = 0
    vx[0] = v0
    vy[0] = 0
    t[0] = 0
    
    # define a function for the right-hand-side:
    def f(vx, vy, x, y, t):
        return (-q*vy*B/m, +q*vx*B/m, vx, vy)
    
    # Calculate the solution using the Midpoint Method:
    for i in range(N-1):
        s1_vx, s1_vy, s1_x, s1_y = f(vx[i], vy[i], x[i], y[i], t[i])
        vx_1 = vx[i] + s1_vx*dt/2
        vy_1 = vy[i] + s1_vy*dt/2
        x_1  = x[i]  + s1_x*dt/2
        y_1  = y[i]  + s1_y*dt/2
        
        s2_vx, s2_vy, s2_x, s2_y = f(vx_1, vy_1, x_1, y_1, t[i]+dt/2)
        vx[i+1] = vx[i] + s2_vx*dt
        vy[i+1] = vy[i] + s2_vy*dt
        x[i+1]  = x[i]  + s2_x*dt
        y[i+1]  = y[i]  + s2_y*dt
        
        t[i+1]  = t[i] + dt
    
    return x, y, t

In [ ]:
fig, axes = plt.subplots(figsize=(12,6))

x, y, t = solve_2(dt=1e-11, tmax=1e-6)

# plot the x and y component vs time
plt.plot(t*1e6, x*100)
plt.plot(t*1e6, y*100)   
plt.title('Trajectory of a proton in a magnetic field')
plt.ylabel('x or y (cm)')
plt.xlabel('t ($\mu$s)')

# plot the trajectory in an inset axis
fig.add_axes([0.68, 0.2, 0.2, 0.4])
plt.plot(x,y)
plt.xlim(-0.18, 0.18)
plt.ylim(-0.04, 0.32)
plt.xlabel('x')
plt.ylabel('y')
plt.xticks([-0.1, 0.0, 0.1])
plt.yticks([ 0.0, 0.1, 0.2])
plt.show()

### Exercise
> Compare the above code with the midpoint scheme with Euler's method. 

> How large can the time step be made while still getting a convergent solution?

> Implement a third solver using a 4th order Runge-Kutta scheme.

In [ ]:
def solve_3(tmax = 1e-6, dt = 1e-11):  
    
    # Allocate memory for arrays and set to zero
    N = round(tmax/dt)
    x = np.zeros(N)
    y = np.zeros(N)
    vx = np.zeros(N)
    vy = np.zeros(N)
    t = np.zeros(N)

    # Initial values
    x[0] = 0
    y[0] = 0
    vx[0] = v0
    vy[0] = 0
    t[0] = 0
    
    # define a function for the right-hand-side:
    def f(vx, vy, x, y, t):
        return (-q*vy*B/m, +q*vx*B/m, vx, vy)
    
    # create an array for the time variable
    t = np.arange(0, tmax, dt)
    
    # Calculate the solution using the RK4 Method:
    for i in range(N-1):
        # s1 = f(y[i], t[i])
        # y1 = y[i] + s1 * dt/2
        
        
        # s2 = f(y1, t[i] + dt/2)
        # y2 = y[i] + s2 * dt/2

        
        # s3 = f(y2, t[i] + dt/2)
        # y3 = y[i] + s3 * dt

        
        # s4 = f(y3, t[i] + dt)
        # y[i+1] = y[i] + (s1 + 2*s2 + 2*s3 + s4)/6*dt


        t[i+1]  = t[i] + dt
    
    return x, y, t

In [ ]:
fig, axes = plt.subplots(figsize=(12,6))

x, y, t = solve_3(dt=1e-11, tmax=1e-6)

# plot the x and y component vs time
plt.plot(t*1e6, x*100)
plt.plot(t*1e6, y*100)   
plt.title('Trajectory of a proton in a magnetic field')
plt.ylabel('x or y (cm)')
plt.xlabel('t ($\mu$s)')

# plot the trajectory in an inset axis
fig.add_axes([0.68, 0.2, 0.2, 0.4])
plt.plot(x,y)
plt.xlim(-0.18, 0.18)
plt.ylim(-0.04, 0.32)
plt.xlabel('x')
plt.ylabel('y')
plt.xticks([-0.1, 0.0, 0.1])
plt.yticks([ 0.0, 0.1, 0.2])
plt.show()

## Solution for the RK4 solver:

In [ ]:
def solve_3_soln(tmax = 1e-6, dt = 1e-11):  
    
    # Allocate memory for arrays and set to zero
    N = round(tmax/dt)
    x = np.zeros(N)
    y = np.zeros(N)
    vx = np.zeros(N)
    vy = np.zeros(N)
    t = np.zeros(N)

    # Initial values
    x[0] = 0
    y[0] = 0
    vx[0] = v0
    vy[0] = 0
    t[0] = 0
    
    # define a function for the right-hand-side:
    def f(vx, vy, x, y, t):
        return (-q*vy*B/m, +q*vx*B/m, vx, vy)
    
    # create an array for the time variable
    t = np.arange(0, tmax, dt)
    
    # Calculate the solution using the RK4 Method:
    for i in range(N-1):
        # s1 = f(y[i], t[i])
        # y1 = y[i] + s1 * dt/2
        s1_vx, s1_vy, s1_x, s1_y = f(vx[i], vy[i], x[i], y[i], t[i])
        vx_1 = vx[i] + s1_vx*dt/2
        vy_1 = vy[i] + s1_vy*dt/2
        x_1  = x[i]  + s1_x*dt/2
        y_1  = y[i]  + s1_y*dt/2
        
        # s2 = f(y1, t[i] + dt/2)
        # y2 = y[i] + s2 * dt/2
        s2_vx, s2_vy, s2_x, s2_y = f(vx_1, vy_1, x_1, y_1, t[i]+dt/2)
        vx_2 = vx[i] + s2_vx*dt/2
        vy_2 = vy[i] + s2_vy*dt/2
        x_2  = x[i]  + s2_x*dt/2
        y_2  = y[i]  + s2_y*dt/2
        
        # s3 = f(y2, t[i] + dt/2)
        # y3 = y[i] + s3 * dt
        s3_vx, s3_vy, s3_x, s3_y = f(vx_2, vy_2, x_2, y_2, t[i]+dt/2)
        vx_3 = vx[i] + s3_vx*dt
        vy_3 = vy[i] + s3_vy*dt
        x_3  = x[i]  + s3_x*dt
        y_3  = y[i]  + s3_y*dt
        
        # s4 = f(y3, t[i] + dt)
        # y[i+1] = y[i] + (s1 + 2*s2 + 2*s3 + s4)/6*dt
        s4_vx, s4_vy, s4_x, s4_y = f(vx_3, vy_3, x_3, y_3, t[i]+dt)
        vx[i+1] = vx[i] + (s1_vx + 2*s2_vx + 2*s3_vx + s4_vx)/6*dt
        vy[i+1] = vy[i] + (s1_vy + 2*s2_vy + 2*s3_vy + s4_vy)/6*dt
        x[i+1]  = x[i]  + (s1_x + 2*s2_x + 2*s3_x + s4_x)/6*dt
        y[i+1]  = y[i]  + (s1_y + 2*s2_y + 2*s3_y + s4_y)/6*dt

        t[i+1]  = t[i] + dt
    
    return x, y, t